<a href="https://colab.research.google.com/github/DanielCalimayor/DS-Unit-2-Regression-2/blob/master/Daniel_Calimayor_Cross_Validation%2C_Hyperparameter_Optimization_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install eli5 pdpbox category_encoders

In [0]:
 # from google.colab import files
 # files.upload()

In [0]:
 #!unzip caterpillar-tube-pricing.zip

In [0]:
 #!unzip data.zip

In [53]:
from glob import glob
import pandas as pd

def search_column(name):
    for path in glob('competition_data/*.csv'):
        df = pd.read_csv(path)
        if name in df.columns:
            print(path, df.shape)
            print(df.columns.tolist(), '\n')
search_column('thickness')

competition_data/comp_elbow.csv (178, 16)
['component_id', 'component_type_id', 'bolt_pattern_long', 'bolt_pattern_wide', 'extension_length', 'overall_length', 'thickness', 'drop_length', 'elbow_angle', 'mj_class_code', 'mj_plug_class_code', 'plug_diameter', 'groove', 'unique_feature', 'orientation', 'weight'] 

competition_data/comp_straight.csv (361, 12)
['component_id', 'component_type_id', 'bolt_pattern_long', 'bolt_pattern_wide', 'head_diameter', 'overall_length', 'thickness', 'mj_class_code', 'groove', 'unique_feature', 'orientation', 'weight'] 

competition_data/comp_float.csv (16, 7)
['component_id', 'component_type_id', 'bolt_pattern_long', 'bolt_pattern_wide', 'thickness', 'orientation', 'weight'] 

competition_data/comp_tee.csv (4, 14)
['component_id', 'component_type_id', 'bolt_pattern_long', 'bolt_pattern_wide', 'extension_length', 'overall_length', 'thickness', 'drop_length', 'mj_class_code', 'mj_plug_class_code', 'groove', 'unique_feature', 'orientation', 'weight'] 



In [0]:
#pip install txt_when_done

In [54]:
import pandas as pd
mats = pd.read_csv('competition_data/bill_of_materials.csv')

ac = mats.melt(id_vars='tube_assembly_id', 
                                     value_vars=[f'component_id_{n}' for n in range(1,9)])

ac = (ac
                       .sort_values(by='tube_assembly_id')
                       .dropna()
                       .rename(columns={'value': 'component_id'}))

ac.head(10)

,tube_assembly_id,variable,component_id
0,TA-00001,component_id_1,C-1622
21198,TA-00001,component_id_2,C-1629
1,TA-00002,component_id_1,C-1312
2,TA-00003,component_id_1,C-1312
3,TA-00004,component_id_1,C-1312
4,TA-00005,component_id_1,C-1624
42400,TA-00005,component_id_3,C-1641
21202,TA-00005,component_id_2,C-1631
5,TA-00006,component_id_1,C-1624
21203,TA-00006,component_id_2,C-1631


In [55]:
comp = pd.read_csv('competition_data/components.csv')
act = ac.merge(comp, how='left')
act.head(3)

,tube_assembly_id,variable,component_id,name,component_type_id
0,TA-00001,component_id_1,C-1622,NUT-SWIVEL,CP-025
1,TA-00001,component_id_2,C-1629,SLEEVE-ORFS,CP-024
2,TA-00002,component_id_1,C-1312,NUT-FLARED,CP-028


In [56]:
table = pd.crosstab(act['tube_assembly_id'], 
                    act['component_type_id'])

table = table.reset_index()
table.columns.name = ''
print(table.shape)
table.head()

(19149, 30)


,tube_assembly_id,CP-001,CP-002,CP-003,CP-004,CP-005,CP-006,CP-007,CP-008,CP-009,CP-010,CP-011,CP-012,CP-014,CP-015,CP-016,CP-017,CP-018,CP-019,CP-020,CP-021,CP-022,CP-023,CP-024,CP-025,CP-026,CP-027,CP-028,CP-029,OTHER
0,TA-00001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
1,TA-00002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,TA-00003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,TA-00004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,TA-00005,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [57]:
comp_float = pd.read_csv('competition_data/comp_float.csv')
comp_float['thickness'].value_counts()

22.00    3
28.00    3
25.40    2
15.70    1
15.85    1
36.00    1
14.20    1
36.50    1
25.00    1
20.00    1
18.00    1
Name: thickness, dtype: int64

In [58]:
comp_float.head()

,component_id,component_type_id,bolt_pattern_long,bolt_pattern_wide,thickness,orientation,weight
0,C-0027,CP-021,148.00,96.00,18.00,Yes,2.230
1,C-0454,CP-022,58.72,30.18,28.00,No,0.590
2,C-0455,CP-022,58.72,30.18,28.00,No,0.525
3,C-0494,CP-022,52.40,26.20,15.85,No,0.230
4,C-0496,CP-022,58.80,30.20,14.20,No,0.284


In [59]:
comp_float['weight']

0     2.230
1     0.590
2     0.525
3     0.230
4     0.284
5     1.144
6     0.447
7     1.983
8     0.465
9     2.565
10    4.060
11    1.880
12    0.783
13    0.276
14    0.389
15    0.516
Name: weight, dtype: float64

In [60]:
comp_float.head()

,component_id,component_type_id,bolt_pattern_long,bolt_pattern_wide,thickness,orientation,weight
0,C-0027,CP-021,148.00,96.00,18.00,Yes,2.230
1,C-0454,CP-022,58.72,30.18,28.00,No,0.590
2,C-0455,CP-022,58.72,30.18,28.00,No,0.525
3,C-0494,CP-022,52.40,26.20,15.85,No,0.230
4,C-0496,CP-022,58.80,30.20,14.20,No,0.284


In [0]:
comp = pd.concat((pd.read_csv(path) for path in glob('competition_data/comp_*.csv')), sort=False)
columns = ['component_id', 'component_type_id', 'orientation', 'unique_feature', 'weight']
comp = comp[columns]
comp['orientation'] = (comp['orientation']=='Yes').astype(int)
comp['unique_feature'] = (comp['unique_feature']=='Yes').astype(int)
comp['weight'] = comp['weight'].fillna(comp['weight'].median())

In [62]:
comp.head()

,component_id,component_type_id,orientation,unique_feature,weight
0,C-0007,CP-014,0,0,0.028
1,C-0030,CP-015,1,0,0.914
2,C-0041,CP-014,0,0,0.120
3,C-0043,CP-014,0,0,0.090
4,C-0044,CP-014,0,0,0.182


In [0]:
mats['orientation'] = mats.rename(columns={'orientation':'orientation_1'}, inplace=True)

In [64]:
comp_float.head()

,component_id,component_type_id,bolt_pattern_long,bolt_pattern_wide,thickness,orientation,weight
0,C-0027,CP-021,148.00,96.00,18.00,Yes,2.230
1,C-0454,CP-022,58.72,30.18,28.00,No,0.590
2,C-0455,CP-022,58.72,30.18,28.00,No,0.525
3,C-0494,CP-022,52.40,26.20,15.85,No,0.230
4,C-0496,CP-022,58.80,30.20,14.20,No,0.284


In [0]:
mats['components_total'] = sum(mats[f'quantity_{n}'].fillna(0)  for n in range(1,9))
mats['components_distinct'] = sum(mats[f'component_id_{n}'].notnull().astype(int) for n in range(1,9))
mats['orientation'] = 0
mats['unique_feature'] = 0
mats['weight'] = 0

for n in range(1,9):
    mats = mats.merge(comp, how='left', 
                                left_on=f'component_id_{n}', 
                                right_on='component_id', 
                                suffixes=('', f'_{n}'))

for col in mats:
    if 'orientation' in col or 'unique_feature' in col or 'weight' in col:
        mats[col] = mats[col].fillna(0)
        
mats['orientation'] = sum(mats[f'orientation_{n}'] for n in range(1,9))
mats['unique_feature'] = sum(mats[f'unique_feature_{n}'] for n in range(1,9))
mats['weight'] = sum(mats[f'weight_{n}'] for n in range(1,9))

features = ['tube_assembly_id', 'orientation', 'unique_feature', 'weight', 
            'components_total', 'components_distinct', 'component_id_1']
mats = mats[features]


In [67]:
comp_float['bolt_pattern_area'] = (comp_float[f'bolt_pattern_long']/ comp_float[f'bolt_pattern_wide'].fillna(0))
comp_float['bolt_pattern_long'] = (comp_float[f'bolt_pattern_long'].fillna(0)) 
comp_float['bolt_pattern_wide'] = (comp_float[f'bolt_pattern_wide'].fillna(0))
comp_float['thickness'] = (comp_float[f'thickness'].fillna(0))



comp_float = comp_float.merge(comp, left_on=f'component_id', right_on='component_id',how='left')

for col in comp_float:
    if 'orientation' in col or 'weight' in col:
        comp_float[col] = comp_float[col].fillna(0)
        
comp_float['orientation'] = (comp_float['orientation'])
comp_float['weight'] = (comp_float['weight'])
comp_float['bolt_pattern_area'] = (comp_float[f'bolt_pattern_long'] * comp_float[f'bolt_pattern_wide'])

features = ['component_id',	'bolt_pattern_long',	'bolt_pattern_wide', 'bolt_pattern_area',
            'thickness',	'orientation',	'weight']
comp_float = comp_float[features]
print(comp_float.shape)
comp_float.head()

(16, 7)


,component_id,bolt_pattern_long,bolt_pattern_wide,bolt_pattern_area,thickness,orientation,weight
0,C-0027,148.00,96.00,14208.0000,18.00,1,2.230
1,C-0454,58.72,30.18,1772.1696,28.00,0,0.590
2,C-0455,58.72,30.18,1772.1696,28.00,0,0.525
3,C-0494,52.40,26.20,1372.8800,15.85,0,0.230
4,C-0496,58.80,30.20,1775.7600,14.20,0,0.284


In [68]:
tube = pd.read_csv('competition_data/tube.csv')
tube.head()

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,SP-0035,12.70,1.65,164.0,5,38.10,N,N,N,N,EF-003,EF-003,0,0,0
1,TA-00002,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0
2,TA-00003,SP-0019,6.35,0.71,127.0,7,19.05,N,N,N,N,EF-008,EF-008,0,0,0
3,TA-00004,SP-0019,6.35,0.71,137.0,9,19.05,N,N,N,N,EF-008,EF-008,0,0,0
4,TA-00005,SP-0029,19.05,1.24,109.0,4,50.80,N,N,N,N,EF-003,EF-003,0,0,0


In [69]:
from sklearn.model_selection import train_test_split

trainval = pd.read_csv('competition_data/train_set.csv')
test = pd.read_csv('competition_data/test_set.csv')

trainval_tube_assemblies = trainval['tube_assembly_id'].unique()
train_tube_assemblies, val_tube_assemblies = train_test_split(
    trainval_tube_assemblies, random_state=42)
train = trainval[trainval.tube_assembly_id.isin(train_tube_assemblies)]
val = trainval[trainval.tube_assembly_id.isin(val_tube_assemblies)]

def clean(X):
    X = X.copy()
    
    
    X['quote_date'] = pd.to_datetime(X['quote_date'], infer_datetime_format=True)
    X['quote_date_year'] = X['quote_date'].dt.year
    X['quote_date_month'] = X['quote_date'].dt.month
    X = X.drop(columns='quote_date')
    
    X = (X.merge(table, how='left')
         .merge(mats, how='left')
         .merge(tube, how='left')
         .merge(comp_float, how='left')
         .fillna(0))
    
    
    
    return X

train = clean(train)
val = clean(val)
test = clean(test)
print(train.shape, val.shape, test.shape)

(22628, 64) (7585, 64) (30235, 64)


In [70]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

target = 'cost'
features = train.columns.drop([target, 'tube_assembly_id'])

X_train = train[features]
X_test = test[features]
y_train = train[target]
y_val = val[target]
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)


y_pred = np.full_like(y_val, fill_value=y_train.mean())
print('Validation RMSLE, Mean Baseline:', rmsle(y_val, y_pred))

Validation RMSLE, Mean Baseline: 0.9418101276064408


In [74]:
!pip install category_encoders

In [0]:
import category_encoders as ce

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
)

In [0]:
from sklearn.model_selection import cross_val_score

In [80]:
%%time 

kev = 3
groups = train['tube_assembly_id']
scores = cross_val_score(pipeline, X_train, y_train_log, cv=kev,
                         scoring='neg_mean_squared_error', groups=groups)
print(f'RMSLE for {kev} folds:', np.sqrt(-scores))

RMSLE for 3 folds: [0.30361072 0.32467543 0.33089356]
CPU times: user 2.48 s, sys: 478 ms, total: 2.96 s
Wall time: 28.8 s


In [81]:
pipeline.named_steps['randomforestregressor']

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [82]:
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestRegressor(random_state=42)
)

param_distributions = {
    'randomforestregressor__n_estimators': randint(50, 500),
    'randomforestregressor__max_features': uniform(),
    'randomforestregressor__min_samples_leaf': [1, 10, 100]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=15,
    cv=2,
    scoring='neg_mean_squared_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train_log, groups=groups)

Fitting 2 folds for each of 15 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   15.5s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.4min finished


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('randomforestregressor',
                                              RandomForestRegressor(bootstrap=True,
                                                                    criterion='mse',
                                    

In [83]:
from xgboost import XGBRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    XGBRegressor(random_state=42)
)

param_distributions = {
    'xgbregressor__n_estimators': randint(500, 1000),
    'xgbregressor__max_depth': randint(3, 7)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=2,
    scoring='neg_mean_squared_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train_log, groups=groups)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  9.8min finished


[14:43:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('xgbregressor',
                                              XGBRegressor(base_score=0.5,
                                                           booster='gbtree',
                                                           col

In [84]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation RMSLE', np.sqrt(-search.best_score_))

Best hyperparameters {'xgbregressor__max_depth': 5, 'xgbregressor__n_estimators': 600}
Cross-validation RMSLE 0.31495283556515224


In [85]:
final_df = pd.DataFrame(search.cv_results_)
final_df.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgbregressor__max_depth,param_xgbregressor__n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
4,25.073543,0.149471,0.456922,0.023310,5,600,"{'xgbregressor__max_depth': 5, 'xgbregressor__...",-0.089788,-0.108602,-0.099195,0.009407,1,-0.011702,-0.005905,-0.008803,0.002898
17,26.526738,0.012387,0.482568,0.020257,5,639,"{'xgbregressor__max_depth': 5, 'xgbregressor__...",-0.089784,-0.108921,-0.099352,0.009569,2,-0.010924,-0.005367,-0.008145,0.002778
5,31.365714,0.143897,0.564729,0.030081,5,753,"{'xgbregressor__max_depth': 5, 'xgbregressor__...",-0.089562,-0.109152,-0.099357,0.009795,3,-0.008978,-0.004251,-0.006614,0.002364
6,29.717713,0.012545,0.537718,0.024891,5,716,"{'xgbregressor__max_depth': 5, 'xgbregressor__...",-0.089805,-0.108992,-0.099399,0.009594,4,-0.009678,-0.004543,-0.007110,0.002567
1,29.730192,0.068954,0.659853,0.025657,5,721,"{'xgbregressor__max_depth': 5, 'xgbregressor__...",-0.089817,-0.109034,-0.099426,0.009609,5,-0.009648,-0.004483,-0.007066,0.002583
3,39.638000,0.120493,0.893732,0.146969,5,963,"{'xgbregressor__max_depth': 5, 'xgbregressor__...",-0.089048,-0.110015,-0.099531,0.010484,6,-0.006506,-0.002985,-0.004746,0.001760
11,41.159186,0.085870,0.807926,0.051081,5,998,"{'xgbregressor__max_depth': 5, 'xgbregressor__...",-0.089061,-0.110008,-0.099534,0.010473,7,-0.006251,-0.002811,-0.004531,0.001720
0,28.731518,0.075577,0.668506,0.031500,6,576,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.088926,-0.111710,-0.100318,0.011392,8,-0.005592,-0.002449,-0.004021,0.001571
9,31.384920,0.021319,0.621338,0.030320,6,633,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.089006,-0.111919,-0.100463,0.011456,9,-0.004813,-0.002077,-0.003445,0.001368
12,43.263140,0.162157,0.957931,0.047545,6,878,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.088836,-0.112476,-0.100656,0.011820,10,-0.002931,-0.001043,-0.001987,0.000944
